## Importing Dependencies

In [3]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25855 sha256=d0eb0ef1d6c246fe97437f52a3b9351e5bdfcf502261e825fd418c027f5a148d
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\31\d0\2c\87ce38f6052879e5b7b18f0f8b4a10ad2a9d210e908d449f16
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split    
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

## Initializing Directory and Categories

In [3]:
DIRECTORY = r"D:\Rishabh\coding\Deep Learning Projects\Face-Mask-Detection\dataset"
CATEGORIES  =['with_mask', 'without_mask']

### grab the list of images in the data and then initialize the list of images and class images 

In [4]:
data = []
labels = []

In [5]:
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)
        
        data.append(image)
        labels.append(category)

c:\Users\Lenovo\anaconda3\envs\MachineLearning\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


## perform encoding on the labels

In [8]:
lb = LabelBinarizer()

labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [9]:
data = np.array(data, dtype='float32')
labels = np.array(labels)

## Splitting the Data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3066, 224, 224, 3) (767, 224, 224, 3) (3066, 2) (767, 2)


## Constructing training image generator for Data Augmentation

In [11]:
aug = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'
)

### Loading MobileNetV2

#### make sure that the head is left off

In [12]:
baseModel = MobileNetV2(weights='imagenet', include_top=False,
                        input_tensor = Input(shape=(224, 224, 3)))

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_3540\3045129431.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights='imagenet', include_top=False,


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


#### Construct the head of the model that will be places on top of the base model

In [15]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name = 'flatten')(headModel)
headModel = Dense(128, activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation='softmax')(headModel)

#### place the head model on the top of the base model(this is the actual model we will train)

In [16]:
model = Model(inputs=baseModel.input, outputs=headModel)

#### loop over all the layers in the base model and freeze them so the will not be updated during the first training process

In [17]:
for layer in baseModel.layers:
    layer.trainable = False

#### Compliling the Model

In [18]:
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

opt = Adam(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)

model.compile(loss='binary_crossentropy', optimizer=opt,
              metrics=['accuracy'])

c:\Users\Lenovo\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


#### Training the head of the network

In [20]:
H = model.fit(
    aug.flow(X_train, y_train, batch_size=BS),
    steps_per_epoch = len(X_train) // BS,
    validation_data = (X_test, y_test),
    validation_steps = len(X_test) // BS,
    epochs = EPOCHS
)

Epoch 1/20


c:\Users\Lenovo\anaconda3\envs\MachineLearning\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


95/95 ━━━━━━━━━━━━━━━━━━━━ 1542s 16s/step - accuracy: 0.7242 - loss: 0.5809 - val_accuracy: 0.9831 - val_loss: 0.1468
Epoch 2/20
 1/95 ━━━━━━━━━━━━━━━━━━━━ 50s 536ms/step - accuracy: 0.8750 - loss: 0.2386

c:\Users\Lenovo\anaconda3\envs\MachineLearning\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8750 - loss: 0.2386   
Epoch 3/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 614s 6s/step - accuracy: 0.9564 - loss: 0.1708 - val_accuracy: 0.9883 - val_loss: 0.0755
Epoch 4/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9375 - loss: 0.1771    
Epoch 5/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 578s 6s/step - accuracy: 0.9728 - loss: 0.1116 - val_accuracy: 0.9909 - val_loss: 0.0538
Epoch 6/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 546us/step - accuracy: 0.9688 - loss: 0.0942 
Epoch 7/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.9751 - loss: 0.0845 - val_accuracy: 0.9935 - val_loss: 0.0449
Epoch 8/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 1.0000 - loss: 0.0479   
Epoch 9/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 82s 844ms/step - accuracy: 0.9823 - loss: 0.0618 - val_accuracy: 0.9935 - val_loss: 0.0384
Epoch 10/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 733us/step - accuracy: 0.9688 - loss: 0.0618 
Epoch 11/20
95/95 ━━━━━━━━━━━━━━━━━━━━ 82s 846ms/step - accuracy

#### Predictions

In [21]:
predictions = model.predict(X_test, batch_size=BS)

24/24 ━━━━━━━━━━━━━━━━━━━━ 17s 621ms/step


In [22]:
predictions = np.argmax(predictions, axis=1)

In [24]:
print(classification_report(y_test.argmax(axis=1), predictions, target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [27]:
model.save('mask_detector.keras', save_format='h5')